# Train latest YOLO model

In [ ]:
from ultralytics import YOLO
# Automatically downloads and loads YOLOv11 model
model = YOLO('yolo11n.pt')

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from src.config import *
from tqdm.auto import tqdm
import os

In [ ]:
data_yaml = "yolo_train.yaml"

args = {
    "task": "detection",
    "mode": "train",
    "epochs": EPOCHS,
    "imgsz": IMGSZ,
    "patience": PATIENCE,
    "project": PROJECT,
    "deterministic": DETERMINISTIC,
    "optimizer": OPTIMIZER,   
    "cos_lr": COS_LR,
    "lr0": LR0,
    "lrf": LRF,
    "momentum": MOMENTUM,
    "box": BOX,
    "cls": CLS,
    "weight_decay": WEIGHT_DECAY,
    "warmup_epochs": WARMUP_EPOCHS,
    "warmup_momentum": WARMUP_MOMENTUM,
    "auto_augment": AUTO_AUGMENT,
    "close_mosaic": CLOSE_MOSAIC,
    "mosaic": MOSAIC,
    "mixup": MIXUP,
    "copy_paste": COPY_PASTE
}

In [ ]:
# Train YOLO model
local_device = model.device
print("Training model on device: ", local_device)
results = model.train(data = data_yaml, **args, device=local_device)
print("Training complete. Model saved.")

In [ ]:
local_device = model.device
print("Training model on device: ", local_device)
results = model.train(
    data=data_yaml, 
    model="src/models/best/best_yolo11x.pt", 
    **args, device=local_device)

### Visualise Training Results

In [ ]:
# Load the CSV file
results_csv = "runs/train/train1/results.csv"  # Path to your results.csv
results_df = pd.read_csv(results_csv)

# Extract metrics
epochs = results_df['epoch']
precision = results_df['metrics/precision(B)']
recall = results_df['metrics/recall(B)']
f1_score = 2 * (precision * recall) / (precision + recall)

# Plot Precision, Recall, and F1-Score
plt.figure(figsize=(10, 6))
plt.plot(epochs, precision, label='Precision', marker='o')
plt.plot(epochs, recall, label='Recall', marker='o')
plt.plot(epochs, f1_score, label='F1-Score', marker='o')
plt.xlabel('Epoch')
plt.ylabel('Metric')
plt.title('Precision, Recall, and F1-Score Over Epochs')
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
def calculate_iou(box1, box2):
    """Calculate Intersection over Union (IoU) between two boxes."""
    x1, y1, x2, y2 = box1
    x1g, y1g, x2g, y2g = box2

    xi1, yi1 = max(x1, x1g), max(y1, y1g)
    xi2, yi2 = min(x2, x2g), min(y2g, y2g)
    inter_area = max(0, xi2 - xi1) * max(0, yi2 - yi1)

    box1_area = (x2 - x1) * (y2 - y1)
    box2_area = (x2g - x1g) * (y2g - y1g)
    union_area = box1_area + box2_area - inter_area

    return inter_area / union_area

In [ ]:
# Load the trained model
model = YOLO("runs/train/train1/weights/best.pt")
test_images_dir = "data/images/val"
ground_truth_dir = "data/labels/val"

results = model.predict(source=test_images_dir, save=False)
total_tp, total_fp, total_fn = 0, 0, 0

# Process each prediction
for result in results:
    img_path = result.path
    gt_file = os.path.join(ground_truth_dir, os.path.splitext(os.path.basename(img_path))[0] + ".txt")

    # Load ground truth boxes
    ground_truth_boxes = []
    with open(gt_file, "r") as f:
        for line in f.readlines():
            _, x_center, y_center, width, height = map(float, line.strip().split())
            img_h, img_w = result.orig_shape
            x1 = int((x_center - width / 2) * img_w)
            y1 = int((y_center - height / 2) * img_h)
            x2 = int((x_center + width / 2) * img_w)
            y2 = int((y_center + height / 2) * img_h)
            ground_truth_boxes.append([x1, y1, x2, y2])

    # Compare predictions to ground truth
    tp, fp, fn = 0, 0, len(ground_truth_boxes)  # Start with all ground truth as false negatives
    for box, conf, cls in zip(result.boxes.xyxy, result.boxes.conf, result.boxes.cls):
        pred_box = [int(coord) for coord in box.tolist()]
        matched = False
        for gt_box in ground_truth_boxes:
            iou = calculate_iou(pred_box, gt_box)
            if iou > 0.5:
                matched = True
                tp += 1
                fn -= 1
                ground_truth_boxes.remove(gt_box)
                break
        if not matched:
            fp += 1

    total_tp += tp
    total_fp += fp
    total_fn += fn

print(f"True Positives (TP): {total_tp}")
print(f"False Positives (FP): {total_fp}")
print(f"False Negatives (FN): {total_fn}")

precision = total_tp / (total_tp + total_fp) if total_tp + total_fp > 0 else 0
recall = total_tp / (total_tp + total_fn) if total_tp + total_fn > 0 else 0
f1_score = 2 * precision * recall / (precision + recall) if precision + recall > 0 else 0
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1-Score: {f1_score:.4f}")